In [ ]:
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


# Define the path to the folder containing the XML annotation files and images.
data_dir = "/Users/ismail/Desktop/LAMBTON/Semester3/project/Final/images-2/"

# Define the classes or object labels you want to detect.
classes = ['eat','friends','help','me','play','thankyou','what','where','who','why','Hello','iloveyou','yes']

# Define some constants for image preprocessing.
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

# Lists to store image data and corresponding labels.
images = []
labels = []

# Parse XML annotation files and load image data.
for xml_file in os.listdir(data_dir):
    if xml_file.endswith(".xml"):
        tree = ET.parse(os.path.join(data_dir, xml_file))
        root = tree.getroot()

        # Extract image file path from XML.
        img_path = os.path.join(data_dir, root.find("path").text)

        # Load and preprocess the image.
        img = cv2.imread(img_path)
        img = cv2.resize(img, IMAGE_SIZE)
        img = img / 255.0  # Normalize pixel values to [0, 1]

        images.append(img)

        # Extract and map class labels.
        obj_name = root.find("object").find("name").text
        label = classes.index(obj_name)
        labels.append(label)


: 

In [20]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=10)

In [21]:
# Create a CNN model.
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*IMAGE_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(classes), activation='softmax')
])

# Compile the model.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model.
model.fit(np.array(X_train), np.array(y_train), epochs=20, batch_size=BATCH_SIZE, validation_data=(np.array(X_val), np.array(y_val)))



Epoch 1/20
4/4 [==============================] - 2s 371ms/step - loss: 14.2443 - accuracy: 0.1250 - val_loss: 15.7314 - val_accuracy: 0.0333
Epoch 2/20
4/4 [==============================] - 1s 286ms/step - loss: 11.8409 - accuracy: 0.0583 - val_loss: 10.9085 - val_accuracy: 0.1000
Epoch 3/20
4/4 [==============================] - 1s 304ms/step - loss: 6.0792 - accuracy: 0.1917 - val_loss: 4.4472 - val_accuracy: 0.1333
Epoch 4/20
4/4 [==============================] - 1s 299ms/step - loss: 3.5698 - accuracy: 0.3500 - val_loss: 2.7374 - val_accuracy: 0.2000
Epoch 5/20
4/4 [==============================] - 1s 313ms/step - loss: 1.9525 - accuracy: 0.4083 - val_loss: 2.1461 - val_accuracy: 0.2333
Epoch 6/20
4/4 [==============================] - 1s 342ms/step - loss: 1.6120 - accuracy: 0.6167 - val_loss: 1.9627 - val_accuracy: 0.3333
Epoch 7/20
4/4 [==============================] - 2s 416ms/step - loss: 1.3350 - accuracy: 0.6667 - val_loss: 1.6853 - val_accuracy: 0.3333
Epoch 8/20
4/4 [

In [22]:
model.save("104labels.h5")

In [27]:
import cv2
import numpy as np
from tensorflow import keras

# Load the trained model.
model = keras.models.load_model("104labels.h5")

# Load and preprocess the test image.
test_image_path = "/Users/ismail/Desktop/LAMBTON/Semester3/project/Final/images-2/what_14.jpg"
IMAGE_SIZE = (128, 128)

img = cv2.imread(test_image_path)
img = cv2.resize(img, IMAGE_SIZE)
img = img / 255.0  # Normalize pixel values to [0, 1]

# Reshape the image to match the model's input shape.
input_image = np.expand_dims(img, axis=0)

# Make predictions with the model.
predictions = model.predict(input_image)

print(predictions)
# Interpret the model's predictions.
predicted_class_index = np.argmax(predictions)
print(predicted_class_index)
predicted_class = classes[predicted_class_index]  # Map index to class label

print(f"The model predicts the image contains: {predicted_class}")


1/1 [==============================] - 0s 84ms/step
[[1.4176017e-03 1.8991564e-03 2.9800987e-07 6.9061317e-03 6.2892491e-07
  7.7704614e-04 9.6538550e-01 3.4470166e-05 2.2824025e-02 7.4119185e-04
  1.2577041e-06 2.3008593e-09 1.2664356e-05]]
6
The model predicts the image contains: what


In [17]:
classes

['eat',
 'friends',
 'help',
 'me',
 'play',
 'thankyou',
 'what',
 'where',
 'who',
 'why']